<a href="https://colab.research.google.com/github/aboagyeso/v3_code/blob/master/Copy_of_P42345_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Embedding, Conv1D, MaxPooling1D, GRU
from keras import regularizers
from keras.optimizers import SGD
from keras.optimizers import Adam, RMSprop
from keras.layers import LSTM
import numpy
import csv
import pandas as pd
import hashlib
import random 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

seed = 7
numpy.random.seed(seed)

maxlen = 100
batch_size = 128

dataset = pd.read_csv("https://raw.githubusercontent.com/aboagyeso/LSTM_SMILES/master/Data/smiles_training/v3/P42345.csv", delimiter=",")
#X_train = dataset.iloc[:,0:1].values
#y_train = dataset.iloc[:,1:2].values
X_train = dataset[['smiles']].values
y_train = dataset[['pAc']].values

for p in range (X_train.shape[0]):
  s = X_train[p,0]
  s = s.replace("[nH]","A")
  s = s.replace("Cl","L")
  s = s.replace("Br","R")
  s = s.replace("[C@]","C")
  s = s.replace("[C@@]","C")
  s = s.replace("[C@@H]","C")
  s =[s[i:i+1] for i in range(0,len(s),1)]
  s = " ".join(s)
  X_train[p,0] = s
X_train = X_train[:,0]  
#y_train = y_train[:,0]
X_train = X_train.tolist()


tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=100)
print(X_train)


Using TensorFlow backend.


[[0 0 0 ... 8 8 2]
 [0 0 0 ... 5 1 4]
 [0 0 0 ... 3 2 4]
 ...
 [0 0 0 ... 4 1 1]
 [0 0 0 ... 1 4 8]
 [0 0 0 ... 1 4 8]]


In [2]:
dataset = pd.read_csv("https://raw.githubusercontent.com/aboagyeso/LSTM_SMILES/master/Data/smiles_validation/smilesP42345.csv", delimiter=",")
#X_test = dataset.iloc[:,0:1].values
#y_test = dataset.iloc[:,1:2].values

X_test = dataset[['smiles']].values
y_test = dataset[['pAc']].values




#X_test = dataset.iloc[:,0:1].values
#y_test = dataset.iloc[:,1:2].values

for p in range (X_test.shape[0]):
  s = X_test[p,0]
  s = s.replace("[nH]","A")
  s = s.replace("Cl","L")
  s = s.replace("Br","R")
  s = s.replace("[C@]","C")
  s = s.replace("[C@@]","C")
  s = s.replace("[C@@H]","C")
  s =[s[i:i+1] for i in range(0,len(s),1)]
  s = " ".join(s)
  X_test[p,0] = s
X_test = X_test[:,0]  
#y_test = y_test[:,0]
X_test = X_test.tolist()
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=100)
#print(X_test)
#print(X_test.shape)



model = Sequential()
model.add(Embedding(100, 128, input_length=100))
model.add(LSTM(128))
model.add(Dense(100, activation='tanh'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer=Adam(0.001))


model.fit(X_train, y_train,
          batch_size=128,
          epochs=20)



W0727 16:26:47.673141 140603886143360 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0727 16:26:47.703999 140603886143360 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0727 16:26:47.708006 140603886143360 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0727 16:26:48.075486 140603886143360 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0727 16:26:48.474663 140603886143360 deprecation.py:323

Epoch 1/20
41505/41505 [==============================] - 97s 2ms/step - loss: 2.8204
Epoch 2/20
41505/41505 [==============================] - 95s 2ms/step - loss: 1.3988
Epoch 3/20
41505/41505 [==============================] - 94s 2ms/step - loss: 1.3823
Epoch 4/20
41505/41505 [==============================] - 95s 2ms/step - loss: 1.1284
Epoch 5/20
41505/41505 [==============================] - 95s 2ms/step - loss: 1.0136
Epoch 6/20
41505/41505 [==============================] - 95s 2ms/step - loss: 0.9947
Epoch 7/20
41505/41505 [==============================] - 95s 2ms/step - loss: 0.9718
Epoch 8/20
41505/41505 [==============================] - 95s 2ms/step - loss: 0.9557
Epoch 9/20
41505/41505 [==============================] - 95s 2ms/step - loss: 0.9228
Epoch 10/20
41505/41505 [==============================] - 54s 1ms/step - loss: 0.8749
Epoch 11/20
41505/41505 [==============================] - 44s 1ms/step - loss: 0.8096
Epoch 12/20
41505/41505 [===========================

In [3]:
score = model.evaluate(X_test, y_test,
                            batch_size=128)
print('Test score:', score)

predictions = model.predict(X_test)


mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2_score = r2_score(y_test, predictions)

print(str(mae)+"\t"+str(mse)+"\t"+str(r2_score))

258/258 [==============================] - 0s 911us/step
Test score: 0.32606953290081764
0.4293070574734973	0.32606950603344415	0.7622480621497536
